In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import seaborn as sns
import tensorflow_addons as tfa
import pickle
from tensorflow.keras.utils import image_dataset_from_directory

In [11]:
training_set = image_dataset_from_directory(
    './train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 76515 files belonging to 4 classes.


In [12]:
validation_set = image_dataset_from_directory(
    './val',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 21861 files belonging to 4 classes.


In [13]:
training_set

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [14]:
INPUT_SHAPE = (224, 224, 3)

In [15]:
mobnet = tf.keras.applications.MobileNetV3Large(
    input_shape=INPUT_SHAPE,
    alpha=1.0,
    minimalistic=False,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    classes=1000,
    pooling=None,
    dropout_rate=0.2,
    classifier_activation="softmax",
    include_preprocessing=True,
    # name="MobileNetV3Large",
)

In [16]:
model = tf.keras.models.Sequential()

In [17]:
model.add(tf.keras.Input(shape=INPUT_SHAPE))

In [18]:
model.add(mobnet)

In [19]:
model.add(tf.keras.layers.Dense(units = 4,activation='softmax'))

In [20]:

metrics_list = [
    'accuracy',
    tfa.metrics.F1Score(num_classes=4, average='macro')
]

In [22]:

model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001), loss='categorical_crossentropy', metrics=metrics_list)

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Functiona  (None, 1000)             5507432   
 l)                                                              
                                                                 
 dense (Dense)               (None, 4)                 4004      
                                                                 
Total params: 5,511,436
Trainable params: 5,487,036
Non-trainable params: 24,400
_________________________________________________________________


In [25]:
training_history = model.fit(x=training_set,validation_data=validation_set,epochs=15)

Epoch 1/15
  21/2392 [..............................] - ETA: 7:50 - loss: 1.3805 - accuracy: 0.6533 - f1_score: 0.3594

KeyboardInterrupt: 

In [ ]:
training_history.history

In [ ]:
model.save("Trained_Eye_dIsease_model.h5")

In [ ]:
with open('Training_history.pkl', 'wb') as f:
    pickle.dump(training_history.history, f)

In [ ]:
with open('Training_history.pkl', 'rb') as f:
    load_history = pickle.load(f)
load_history

In [ ]:
epochs = [i for i in range(1,16)]
plt.plot(epochs,load_history['loss'],color='red',label='Training Loss')
plt.plot(epochs,load_history['val_loss'],color='blue',label='Validation Loss')
plt.xlabel('No. of Epochs')
plt.title('Visualization of Loss Result')
plt.legend()
plt.show()

In [ ]:
model = tf.keras.models.load_model("Trained_Eye_dIsease_model.h5")

In [ ]:
model.summary()

In [ ]:
test_set = image_dataset_from_directory(
    './test',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(224, 224),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
test_loss,test_acc,f1_score = model.evaluate(test_set)

In [ ]:
test_loss,test_acc,f1_score 

In [ ]:
true_categories = tf.concat([y for x, y in test_set], axis=0)
Y_true = tf.argmax(true_categories, axis=1)

In [ ]:
true_categories

In [ ]:
Y_true

In [ ]:
predicted_categories = model.predict(test_set)
Y_pred = tf.argmax(predicted_categories, axis=1)

In [ ]:
Y_pred

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_true,Y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_true,Y_pred)
cm

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(cm,annot=True,annot_kws={"size":8})
plt.xlabel("Predicted Class",fontsize=10)
plt.ylabel("Actual Class",fontsize=10)
plt.title("Human Eye Disease Prediction Confusion Matrix",fontsize=12)
plt.show()